In [1]:
import re

import numpy as np
from langchain_core.tools import tool
import requests

import ollama

from crm_chatbot.resource_loader import ResourceLoader

faq_text = ResourceLoader.read_text_resource("swiss_faq.md")

docs = [{"page_content": txt} for txt in re.split(r"(?=\n##)", faq_text)]